In [5]:
import pytest
import ipytest

ipytest.autoconfig()

In [6]:
from ipynb.fs.full.Utility_functions import get_minutes_played
import pandas as pd

In [7]:
@pytest.fixture
def data_test_col():
    return ['match_id', 'team.name', 'player.name', 'type.name',
            'time', 'period', 'substitution.replacement.name',
            'bad_behaviour.card.name', 'foul_committed.card.name',
            'tactics.lineup']


@pytest.fixture
def lineup():
    return """[{'player': {'id': 0000, 'name': 'Robert Pires'},
                 'position': {'id': 16, 'name': 'Left Midfield'},
                 'jersey_number': 7}]"""


@pytest.mark.parametrize("entry_time", [0.0, 10.5, 50.0, float('NaN')])
@pytest.mark.parametrize("period", [1, 2])
@pytest.mark.parametrize("card", ['Second Yellow', 'Red Card'])
@pytest.mark.parametrize("source", ['Bad Behaviour', 'Foul Committed', 'Substitution', ''])
def test_get_minutes_played_starter(data_test_col, lineup, entry_time, period, card, source):
    """Test get_minutes_played() for all possible cases: 
        starter/not starter, finish the game/do not finish the game, also depending on period 1 or 2.

    """
    
    # Fix length of each half
    first_half_duration = 47.5
    second_half_duration = 49
    
    # If entry_time is 50.0, entry is on the second half
    # so sub OFF (if any) shoud be in period 2
    if entry_time == 50.0:
        period = 2
    
    # If player was sent off (from either replacement or red card)
    if source in ['Bad Behaviour', 'Foul Committed', 'Substitution']:
        # Depending on first or second half, fix a different total time
        if period == 1:
            total_minutes = 35.0 - entry_time
            sendoff_time = 35.0
        elif period == 2:
            # Total minutes depends on entry_time (first or second half)
            total_minutes = (int(entry_time in [0.0, 10.5])
                             )*first_half_duration + (int(entry_time == 50.0)
                                                      )*45 + 15.0 - entry_time
            sendoff_time = 60.0
    else:
        # total minutes depends on entry_time (first or second half)
        total_minutes = (int(entry_time in [0.0, 10.5])
                         )*first_half_duration + (int(entry_time == 50.0)
                                                  )*45 + second_half_duration - entry_time
        
    # Create fake data
    data = [
        [0, 'FC_test',  float('NaN'), 'Half End', first_half_duration, 1,
         float('NaN'), float('NaN'), float('NaN'), ""],
        [0, 'FC_test', float('NaN'), 'Half End', 45 + second_half_duration, 2,
         float('NaN'), float('NaN'), float('NaN'), ""],
    ]
    
    # If player is in the Starting XI
    if entry_time == 0.0:
        # Add a lineup with his name to the data
        data.append(
            [0, 'FC_test', float('NaN'), 'Starting XI', 0.0, 1,
             float('NaN'), float('NaN'), float('NaN'), lineup]
                    )
    else:
        # Add lineup without his name to the data
        lineup_without_pires = """[{'player': {'id': 0000, 'name': 'Fabien Barthez'},
                                    'position': {'id': 1, 'name': 'GoalKeeper'},
                                    'jersey_number': 16}]"""
        
        data.append(
            [0, 'FC_test', float('NaN'), 'Starting XI', 0.0, 1,
             float('NaN'), float('NaN'), float('NaN'), lineup_without_pires]
                    )
    
        if entry_time == 10.5:
            # Add a sub ON in the first half to the data 
            data.append(
                [0, 'FC_test', float('NaN'), 'Substitution', entry_time, 1,
                 'Robert Pires', float('NaN'), float('NaN'), ""]
                        )
        elif entry_time == 50.0:
            # Add a sub ON in the second half to the data 
            data.append(
                [0, 'FC_test', float('NaN'), 'Substitution', entry_time, 2,
                 'Robert Pires', float('NaN'), float('NaN'), ""]
                        )
        else:
            # Player never went ON
            total_minutes = 0
            # Player could not be sent OFF so putting source (of sent off) to ''
            source = ''

    
    # Add an entry to the data, who corresponds to 'sent off'
    if source == 'Bad Behaviour':
        data.append(
            [0, 'FC_test', 'Robert Pires', source, sendoff_time, period,
             float('NaN'), card, float('NaN'), ""])
    elif source == 'Foul Committed':
        data.append(
            [0, 'FC_test', 'Robert Pires', source, sendoff_time, period,
             float('NaN'), float('NaN'), card, ""])
    elif source == 'Substitution':
        data.append(
            [0, 'FC_test', 'Robert Pires', source, sendoff_time, period,
             float('NaN'), float('NaN'), float('NaN'), ""])
    else:
        pass

    data_test = pd.DataFrame(data, columns=data_test_col)
    
    total_found = get_minutes_played(data_test, 0, 'Robert Pires', 'FC_test')

    assert total_found == total_minutes, f"""
        Should be equal to total minutes before exclusion: {total_minutes}."""

In [8]:
ipytest.run('-qq')

................................................................         [100%]
=============================== warnings summary ===============================
tmp9v_nj8dp.py: 28 tests with warnings
  /home/ysance.local/benjamin.larrousse/Envs/env_statsbomb_38/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
    res_values = method(rvalues)

-- Docs: https://docs.pytest.org/en/latest/warnings.html
